In [10]:
import sparql_dataframe

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

endpoint = 'https://dati.cultura.gov.it/sparql'

query_culture_events = """
PREFIX cis: <http://dati.beniculturali.it/cis/>
PREFIX arco: <https://w3id.org/arco/ontology/arco/>
PREFIX clvapit:<https://w3id.org/italia/onto/CLV/>

SELECT DISTINCT ?s ?event ?o ?site ?urlcity ?city WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity rdfs:label ?city
}
"""

In [11]:
df = sparql_dataframe.get(endpoint, query_culture_events)

In [12]:
print(df)

                                                       s  \
0      http://dati.beniculturali.it/mibact/eventi/res...   
1      http://dati.beniculturali.it/mibact/eventi/res...   
2      http://dati.beniculturali.it/mibact/eventi/res...   
3      http://dati.beniculturali.it/mibact/eventi/res...   
4      http://dati.beniculturali.it/mibact/eventi/res...   
...                                                  ...   
21163  http://dati.beniculturali.it/mibact/eventi/res...   
21164  http://dati.beniculturali.it/mibact/eventi/res...   
21165  http://dati.beniculturali.it/mibact/eventi/res...   
21166  http://dati.beniculturali.it/mibact/eventi/res...   
21167  http://dati.beniculturali.it/mibact/eventi/res...   

                                                   event  \
0      “Un brindisi per l’eternità”, all'Antiquarium ...   
1      Giornate Europee del Patrimonio 2022 al Museo ...   
2              La storia ragionevole dei Bronzi di Riace   
3      Osservando le mura aureliane: un

In [23]:
df=df.drop_duplicates(["s", "o"])

In [24]:
df.to_csv("/Users/martinapensalfini/Desktop/infowiz/cleancefile.csv")

In [25]:
df_city=df["city"]
print(df_city)

0                    Numana
1                     Lecce
2        Reggio di Calabria
3                      Roma
4        Reggio di Calabria
                ...        
21163               Venezia
21164                 Sarno
21165               Bologna
21166               Bologna
21167               Bologna
Name: city, Length: 21101, dtype: object


In [26]:
counts = dict()
for i in df["city"]:
    if i=="Bologna":
        counts[i] = counts.get(i, 0) + 1
print(counts)


{'Bologna': 198}


In [27]:
counts = dict()
for i in df["city"]:
    if i=="Zogno":
        counts[i] = counts.get(i, 0) + 1
print(counts)

{'Zogno': 2}


In [28]:
counts = dict()
for i in df["city"]:
    if i=="Amalfi":
        counts[i] = counts.get(i, 0) + 1
print(counts)

{'Amalfi': 2}


In [29]:
counts = dict()
for i in df["city"]:
    if i=="Buddusò":
        counts[i] = counts.get(i, 0) + 1
print(counts)

{'Buddusò': 1}


In [ ]:
#to retrieve dbPedia links
query = '''
SELECT DISTINCT(?urlcity2) ?dbCity WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity owl:sameAs ?urlcity2.
?urlcity2 owl:sameAs ?dbCity. 
FILTER(contains (str(?dbCity), "dbpedia") )
}
'''

In [1]:
from sparql_dataframe import get
endpoint = 'https://dati.cultura.gov.it/sparql'

#count of all the cities
query = '''
SELECT COUNT(DISTINCT(?urlcity) as ?count) WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
}
'''
CountCity = get(endpoint, query)
CountCity

,callret-0
0,1018


In [2]:
#count of all the cities
query = '''
SELECT COUNT(DISTINCT(?urlcity2) as ?count) WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity owl:sameAs ?urlcity2.
}
'''
SecondUrlCount = get(endpoint, query)
SecondUrlCount

,callret-0
0,797


In [ ]:
#it works, N.B.: the link of the cities points with owl:sameAs always at another link of mibact of the same city. From this second link it is possible to retrieve the owl:sameAs link to dbpedia.
#Some of the cities (few) have two dbpedia links, one to the city and one to the Province, but filtering them [FILTER(!contains(str(?dbpedia), "Province"))], they are less than the overall number, which means that probably some of them have only the Province link. 
#However, since we want to link the coordinates to the cultural event and not to the city we will leave it like this now.
query = '''
SELECT DISTINCT(?urlcity) ?urlcity2 ?dbpedia WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity owl:sameAs ?urlcity2.
?urlcity2 owl:sameAs ?dbpedia.
FILTER (contains(str(?dbpedia), "dbpedia"))
}
'''

In [2]:
from sparql_dataframe import get
endpoint = 'https://dati.cultura.gov.it/sparql'

query = '''
SELECT DISTINCT(?urlcity) ?dbpedia WHERE {
 ?s a cis:CulturalEvent;
rdfs:label ?event.
?s cis:isHostedBySite ?o.
?o rdfs:label ?site.
?o cis:siteAddress ?address.
?address clvapit:hasCity ?urlcity.
?urlcity owl:sameAs ?urlcity2.
?urlcity2 owl:sameAs ?dbpedia.
FILTER (contains(str(?dbpedia), "dbpedia"))

}
'''

dbPediaLinks = get(endpoint, query)
dbPediaLinks

,urlcity,dbpedia
0,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Besenello
1,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Calci
2,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Carmignano
3,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Cetona
4,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Chiusi
...,...,...
789,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Casalciprano
790,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Monterosso_Grana
791,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Castrocielo
792,http://dati.beniculturali.it/mibact/eventi/res...,http://dbpedia.org/resource/Lu_e_Cuccaro_Monfe...


In [8]:
from pandas import DataFrame, Series
dbpedia = dbPediaLinks["dbpedia"]

link = []

AllCoordinates = DataFrame({"latitude":[], "longitude":[]})

endpoint= 'https://dbpedia.org/sparql'
for idx, item in dbpedia.iteritems():
    query = '''
    SELECT ?lat ?long WHERE {
        <{item}> geo:lat ?lat;
        geo:long ?long.
    } '''
    coordinates = get(endpoint, query)
    link.append(item)
    AllCoordinates.append(coordinates)

AllCoordinates


C:\Users\bianc\AppData\Local\Temp\ipykernel_29852\78134372.py:9: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for idx, item in dbpedia.iteritems():


QueryBadFormed: QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b"Virtuoso 37000 Error SP030: SPARQL compiler, line 4: syntax error at '<' before '{'\n\nSPARQL query:\n#output-format:text/csv\n\n    SELECT ?lat ?long WHERE {\n        <{item}> geo:lat ?lat;\n        geo:long ?long\n    } "